In [2]:
%pip install selenium
%pip install beautifulsoup4

  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ------ --------------------------------- 1.6/9.5 MB 13.9 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 31.2 MB/s eta 0:00:00
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached websocket_client-1.8.0-py3-non

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [95]:
from selenium.webdriver.chrome.service import Service

download_service = Service()

driver = webdriver.Chrome(service=download_service)

#driver.maximize_window()

The chromedriver version (131.0.6778.69) detected in PATH at I:\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.83); currently, chromedriver 132.0.6834.83 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


In [96]:
# Otwieranie strony Steam
url = "https://store.steampowered.com/category/science_fiction/"
driver.get(url)

# Czekanie na załadowanie treści
time.sleep(2)  # Dostosuj, jeśli strona ładuje się szybciej

In [106]:
# zescrolluj na dół
driver.execute_script('window.scroll(0, document.body.scrollHeight-3000)')

In [122]:
driver.execute_script('window.scroll(0, document.body.scrollHeight-1600)')

In [107]:
def GetTitles():
    # Wyciąganie tytułów gier za pomocą XPath
    try:
        # XPath do najbardziej zagnieżdżonych divów z tytułami gier
        title_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]//a//div")
        titles = [title.text.strip() for title in title_elements if title.text.strip()]

        # Wyświetlenie wyników
        print("Lista tytułów gier:")
        for title in titles:
            print(title)

        return titles

    except Exception as e:
        print(f"Wystąpił błąd: {e}")

In [118]:
def GetTitlesWithTags():
    games = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

            # Dodanie gry i jej tagów do listy
            games.append({
                "title": title,
                "tags": tags
            })
        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Wyświetlenie wyników
    print("Lista gier i ich tagów:")
    for game in games:
        print(f"Tytuł: {game['title']}, Tagi: {', '.join(game['tags'])}")

In [120]:
def GetTitlesWithTagsAndPrices():
    games = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = [tag.text.strip() for tag in tag_elements if tag.text.strip()]

            # Pobranie ceny gry
            try:
                price_container = game.find_element(By.XPATH, ".//div[contains(@class, '_1JuIpzMtS7-xZrnUmEQ4my')]//div[contains(@class, '_2s-O5T3qJJYR2AUq4b9jIN')]")
                price = price_container.find_element(By.XPATH, ".//div[contains(@class, '_3j4dI1yA7cRfCvK8h406OB')]").text.strip()
            except Exception as e:
                price = "Brak ceny"  # Obsługa gier bez podanej ceny

            # Dodanie gry do listy
            games.append({
                "title": title,
                "tags": tags,
                "price": price
            })
        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Wyświetlenie wyników
    print("Lista gier, ich tagów i cen:")
    for game in games:
        print(f"Tytuł: {game['title']}, Tagi: {', '.join(game['tags'])}, Cena: {game['price']}")

    return games


In [123]:
GetTitlesWithTagsAndPrices()

Lista gier, ich tagów i cen:
Tytuł: Jurassic World Evolution 2, Tagi: Dinozaury, Budowanie, Symulatory, Zarządzanie, Piaskownica, Cena: 64,49 zł
Tytuł: Lies of P, Tagi: Souls-like, Akcja, Mroczne fantasy, RPG akcji, Eksploracja, Cena: 164,99 zł
Tytuł: Project Zomboid, Tagi: Survival, Zombie, Otwarty świat, Wieloosobowe, Survivalowe w otwartym świecie, Cena: 91,99 zł
Tytuł: Metro Exodus, Tagi: Postapokaliptyczne, FPS, Jednoosobowe, Otwarty świat, Bogata fabuła, Cena: 31,75 zł
Tytuł: Planetary Annihilation: TITANS, Tagi: RTS, Strategiczne, Taktyka w czasie rzeczywistym, Wieloosobowe, Budowanie bazy, Cena: 35,63 zł
Tytuł: The First Descendant, Tagi: Free to Play, Treści seksualne, Strzelanka z łupami, Wieloosobowe, Strzelanka z perspektywą trzeciej osoby, Cena: Free to Play
Tytuł: Once Human, Tagi: Wieloosobowe, Survivalowe w otwartym świecie, Survival, Strategiczne, Horror, Cena: Free to Play
Tytuł: Infection Free Zone, Tagi: Budowanie bazy, Postapokaliptyczne, Budowanie miasta, Strategi

[{'title': 'Jurassic World Evolution 2',
  'tags': ['Dinozaury',
   'Budowanie',
   'Symulatory',
   'Zarządzanie',
   'Piaskownica'],
  'price': '64,49 zł'},
 {'title': 'Lies of P',
  'tags': ['Souls-like',
   'Akcja',
   'Mroczne fantasy',
   'RPG akcji',
   'Eksploracja'],
  'price': '164,99 zł'},
 {'title': 'Project Zomboid',
  'tags': ['Survival',
   'Zombie',
   'Otwarty świat',
   'Wieloosobowe',
   'Survivalowe w otwartym świecie'],
  'price': '91,99 zł'},
 {'title': 'Metro Exodus',
  'tags': ['Postapokaliptyczne',
   'FPS',
   'Jednoosobowe',
   'Otwarty świat',
   'Bogata fabuła'],
  'price': '31,75 zł'},
 {'title': 'Planetary Annihilation: TITANS',
  'tags': ['RTS',
   'Strategiczne',
   'Taktyka w czasie rzeczywistym',
   'Wieloosobowe',
   'Budowanie bazy'],
  'price': '35,63 zł'},
 {'title': 'The First Descendant',
  'tags': ['Free to Play',
   'Treści seksualne',
   'Strzelanka z łupami',
   'Wieloosobowe',
   'Strzelanka z perspektywą trzeciej osoby'],
  'price': 'Free 

In [119]:
GetTitlesWithTags()

Lista gier i ich tagów:
Tytuł: Marvel Rivals, Tagi: Free to Play, Wieloosobowe, Strzelanka z bohaterami, Strzelanka z perspektywą trzeciej osoby, Superbohater
Tytuł: Warhammer 40,000: Space Marine 2, Tagi: Warhammer 40K, Akcja, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Kooperacja
Tytuł: Detroit: Become Human, Tagi: Znaczące wybory, Bogata fabuła, Wiele zakończeń, Filmowe, Jednoosobowe
Tytuł: Apex Legends™, Tagi: Free to Play, Battle royale, Wieloosobowe, FPS, Strzelanka
Tytuł: HELLDIVERS™ 2, Tagi: Sieciowa kooperacja, PvE, Strzelanka z perspektywą trzeciej osoby, Wieloosobowe, Akcja
Tytuł: Among Us, Tagi: Wieloosobowe, Sieciowa kooperacja, Dedukcyjne towarzyskie, Kosmos, Survival
Tytuł: Stellaris, Tagi: Kosmos, Strategiczne globalne, Strategiczne, Science fiction, 4X
Tytuł: Cyberpunk 2077, Tagi: Cyberpunk, Otwarty świat, Nagość, RPG, Jednoosobowe
Tytuł: Frostpunk, Tagi: Budowanie miasta, Survival, Zarządzanie zasobami, Strategiczne, Postapokaliptyczne
Tytuł: They Are Billi

In [115]:
GetTitles()

Lista tytułów gier:
7 Days to Die
Marvel Rivals
Warhammer 40,000: Space Marine 2
Detroit: Become Human
Apex Legends™
HELLDIVERS™ 2
Among Us
Stellaris
Cyberpunk 2077
Frostpunk
They Are Billions
Warframe
ICARUS
Stray
Overwatch® 2
Cyberpunk 2077: Widmo wolności
S.T.A.L.K.E.R. 2: Heart of Chornobyl
Halo: The Master Chief Collection
Frostpunk 2
Lethal Company
BlazBlue Entropy Effect
Atomic Heart
This War of Mine
The Last of Us™ Part II Remastered
RimWorld
Jurassic World Evolution 2
Lies of P
Project Zomboid
Metro Exodus
Planetary Annihilation: TITANS
The First Descendant
Once Human
Infection Free Zone
Sins of a Solar Empire II
Warhammer 40,000: Darktide
Chained Together
Outer Wilds


['7 Days to Die',
 'Marvel Rivals',
 'Warhammer 40,000: Space Marine 2',
 'Detroit: Become Human',
 'Apex Legends™',
 'HELLDIVERS™ 2',
 'Among Us',
 'Stellaris',
 'Cyberpunk 2077',
 'Frostpunk',
 'They Are Billions',
 'Warframe',
 'ICARUS',
 'Stray',
 'Overwatch® 2',
 'Cyberpunk 2077: Widmo wolności',
 'S.T.A.L.K.E.R. 2: Heart of Chornobyl',
 'Halo: The Master Chief Collection',
 'Frostpunk 2',
 'Lethal Company',
 'BlazBlue Entropy Effect',
 'Atomic Heart',
 'This War of Mine',
 'The Last of Us™ Part II Remastered',
 'RimWorld',
 'Jurassic World Evolution 2',
 'Lies of P',
 'Project Zomboid',
 'Metro Exodus',
 'Planetary Annihilation: TITANS',
 'The First Descendant',
 'Once Human',
 'Infection Free Zone',
 'Sins of a Solar Empire II',
 'Warhammer 40,000: Darktide',
 'Chained Together',
 'Outer Wilds']

In [88]:
# zescrolluj na dół
driver.execute_script('window.scroll(0, document.body.scrollHeight-3800)')

GetTitles()

driver.execute_script('window.scroll(0, document.body.scrollHeight-2100)')


Lista tytułów gier:
Warhammer 40,000: Space Marine 2
Marvel Rivals
Warhammer 40,000: Space Marine 2
Detroit: Become Human
Apex Legends™
HELLDIVERS™ 2
Stellaris
Among Us
Cyberpunk 2077
Frostpunk
Warframe
They Are Billions
ICARUS


In [114]:
load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
load_more_button.click()

In [60]:
# Funkcja do przewijania i klikania
def scroll_and_click(driver, scroll_pause=1):
    try:
        # Przewijanie na dół
        driver.execute_script('window.scroll(0, document.body.scrollHeight-2200)')
        time.sleep(scroll_pause)  # Poczekaj na załadowanie
        # Klikanie przycisku
        load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
        load_more_button.click()
        time.sleep(scroll_pause)  # Poczekaj na załadowanie nowych gier
    except Exception as e:
        print(f"Nie udało się kliknąć przycisku: {e}")

# Wyciąganie tytułów gier
def extract_titles(driver):
    try:
        title_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]//a//div")
        titles = [title.text.strip() for title in title_elements if title.text.strip()]  # Pobieranie tekstów tytułów
        return titles
    except Exception as e:
        print(f"Błąd podczas wyciągania tytułów: {e}")
        return []

# Lista do przechowywania wszystkich tytułów
all_titles = []

# Powtarzanie procesu 10 razy
for i in range(2):
    print(f"Iteracja {i+1}/10...")
    # Przewijanie i klikanie
    scroll_and_click(driver)
    # Zbieranie danych
    new_titles = extract_titles(driver)
    all_titles.extend(new_titles)

# Usuwanie duplikatów
unique_titles = list(set(all_titles))

# Wyświetlenie wyników
print("Zebrano unikalne tytuły gier:")
for title in unique_titles:
    print(title)

Iteracja 1/10...
Nie udało się kliknąć przycisku: Message: element click intercepted: Element <button class="_2tkiJ4VfEdI9kq1agjZyNz Focusable">...</button> is not clickable at point (457, 857). Other element would receive the click: <div id="cookiePrefPopup" class="cookiepreferences_popup" style="display: block;">...</div>
  (Session info: chrome=132.0.6834.83)
Stacktrace:
	GetHandleVerifier [0x00007FF67A6C2775+28773]
	(No symbol) [0x00007FF67A62AFB0]
	(No symbol) [0x00007FF67A4C552A]
	(No symbol) [0x00007FF67A520B5E]
	(No symbol) [0x00007FF67A51E5FC]
	(No symbol) [0x00007FF67A51B7E6]
	(No symbol) [0x00007FF67A51A741]
	(No symbol) [0x00007FF67A50C970]
	(No symbol) [0x00007FF67A53EF9A]
	(No symbol) [0x00007FF67A50C1C6]
	(No symbol) [0x00007FF67A53F1B0]
	(No symbol) [0x00007FF67A55F1A4]
	(No symbol) [0x00007FF67A53ED43]
	(No symbol) [0x00007FF67A50A548]
	(No symbol) [0x00007FF67A50B6B1]
	GetHandleVerifier [0x00007FF67A9EF45D+3358029]
	GetHandleVerifier [0x00007FF67AA0430D+3443709]
	GetH

In [ ]:
# Zamknięcie drivera
driver.quit()